In [9]:
#This is a streamlined call that only fetches data for all Batteries mentioned in a list - actual generation in MW h, for each BM Unit separate and with discharge and charge info separately
import requests
import pandas as pd
import os
from datetime import date

# --- Setup ---
start_date = date(2024, 6, 30)
end_date = date(2025, 7, 1)

# Load BMU to fuel type mapping
mapping_path = r"C:\Users\spice\Dropbox\Documents\Imperial 2024.2025\MECH70038 - Research Projects\_My Thesis\Data\BESS_Master_List.csv"
mapping_df = pd.read_csv(mapping_path)

# No filtering – include all BM Units
bm_units = mapping_df['NESO BMU ID'].dropna().unique().tolist()
if not bm_units:
    raise ValueError("No BM Units found in the mapping data.")

# --- API Call ---
base_url = "https://data.elexon.co.uk/bmrs/api/v1/datasets/B1610/stream"
params = {
    "from": start_date.isoformat(),
    "to": end_date.isoformat(),
    "bmUnit": bm_units
}

response = requests.get(base_url, params=params)
response.raise_for_status()
data = response.json()

df = pd.DataFrame(data)
if df.empty:
    raise ValueError("No data returned from B1610 API.")

# --- Merge & Processing ---
df_merged = df.merge(mapping_df, left_on="nationalGridBmUnitId", right_on="NESO BMU ID", how="left")

# Create positive and negative generation
df_merged['positive_generation'] = df_merged['quantity'].apply(lambda x: x if x > 0 else 0)
df_merged['negative_generation'] = df_merged['quantity'].apply(lambda x: x if x < 0 else 0)

# Pivot per BMU
pivot_pos = df_merged.pivot_table(
    index=['settlementDate', 'settlementPeriod'],
    columns='nationalGridBmUnitId',
    values='positive_generation',
    aggfunc='sum'
).add_prefix('pos_')

pivot_neg = df_merged.pivot_table(
    index=['settlementDate', 'settlementPeriod'],
    columns='nationalGridBmUnitId',
    values='negative_generation',
    aggfunc='sum'
).add_prefix('neg_')

# --- Combine & Interleave Columns ---
pivot_df = pd.concat([pivot_pos, pivot_neg], axis=1)

# Extract BMU IDs from the column names
bm_unit_ids = sorted({col.split('_', 1)[1] for col in pivot_df.columns if '_' in col})

# Construct interleaved column order
interleaved_cols = []
for bmu_id in bm_unit_ids:
    interleaved_cols.append(f'pos_{bmu_id}')
    interleaved_cols.append(f'neg_{bmu_id}')

# Add the index columns back at the front
pivot_df = pivot_df.reset_index()
cols_order = ['settlementDate', 'settlementPeriod'] + [col for col in interleaved_cols if col in pivot_df.columns]
pivot_df = pivot_df[cols_order]

# Reconstruct GSP row using the new interleaved order
gsp_row = {'settlementDate': 'GSP Zone', 'settlementPeriod': ''}
bm_ids = mapping_df.set_index("NESO BMU ID")["GSP Zone"].to_dict()
for col in cols_order[2:]:  # Skip settlementDate and settlementPeriod
    bmu_id = col.split('_', 1)[1]
    gsp_row[col] = bm_ids.get(bmu_id, '')

# Insert the GSP row
pivot_df = pd.concat([
    pd.DataFrame([gsp_row]),
    pivot_df
], ignore_index=True)

# --- Save ---
start_str = start_date.strftime("%Y-%m-%d")
end_str = end_date.strftime("%Y-%m-%d")
filename = f"BMunitPerID_{start_str}_to_{end_str}.csv"

output_dir = r"C:\Users\spice\Dropbox\Documents\Imperial 2024.2025\MECH70038 - Research Projects\_My Thesis\Data"
os.makedirs(output_dir, exist_ok=True)
output_file = os.path.join(output_dir, filename)

pivot_df.to_csv(output_file, index=False)
print(f"Saved to {output_file}")


Saved to C:\Users\spice\Dropbox\Documents\Imperial 2024.2025\MECH70038 - Research Projects\_My Thesis\Data\BMunitPerID_2024-06-30_to_2025-07-01.csv
